# 4 Test case

This notebook runs a basic parametrization of the model.

We simulate an SIR epidemic in a system of M subpopulations.



In [1]:
import numpy as np

from EpiCommute import SIRModel

%matplotlib widget

## Initialize model

Here we set the parameters of the model, and use some dummy data

In [2]:
M = 20    # Number of subpopulations

# Initialize a random mobility matrix
mobility = _F = np.array([
            [ 8, 2, 1 ],
            [ 4, 6, 0 ],
            [ 3, 0, 5 ],
        ],dtype=float)
mobility = mobility.T

subpopulation_sizes = _Npop_home = np.array([360.,180.,120.])
T_max = 6
dt = 0.1
dt_save = 1.0
# Initialize the model
model = SIRModel(
            mobility,
            subpopulation_sizes,
            outbreak_source=0, # random outbreak location
            dt=dt,                   # simulation time interval
            dt_save=dt_save,                # time interval when to save observables
            I0=120,                    # number of initial infected
            VERBOSE=False,              # print verbose output
            mu=2,
            R0=2,
            T_max = T_max,
            save_observables=['epi_total'],
        )
model.reset_initialize_simulation()
print(model.population)
print(model.I.sum(axis=1))
print(model.I.sum(axis=1)/subpopulation_sizes.sum())

[[192  96  72]
 [ 45 135   0]
 [ 20   0 100]]
[120   0   0]
[0.18181818 0.         0.        ]


## Run simulation

In [3]:
Nmeas = 100
I = np.zeros_like(model.I)
for meas in range(Nmeas):
    model.reset_initialize_simulation()
    I += model.I
print(I/Nmeas)

_I0 = np.array([
    [ 64, 0., 0.],
    [ 32, 0., 0.],
    [ 24, 0., 0.],
    ]).T
_S0 = np.array([
        [ 128, 45, 20 ],
        [ 64, 135, 0. ],
        [ 48, 0., 100 ],
    ]).T
_R0 = np.zeros_like(_S0)

_beta = 4.
_lambda_home = 0.5*_beta*np.array([ 1/3., 0, 0])
_lambda_work = 0.5*_beta*np.array([ 64/257, 32/231, 24/172 ])

_lambda_ij = 0.5*_beta*np.array([
        [1/3+64/257, 64/257, 64/257,],
        [1/3+32/231, 32/231, 32/231,],
        [1/3+24/172, 24/172, 24/172,]
    ]).T

S, I, R = _S0, _I0, _R0
# Home force of infection
I_ij_sumj = _I0.sum(axis=1)
N_ij_sumj = _S0.sum(axis=1) + _I0.sum(axis=1) + _R0.sum(axis=1)
lambda_home = 0.5 * model.beta * I_ij_sumj / N_ij_sumj
# Work force of infection
I_ji_sumj = _I0.sum(axis=0)
N_ji_sumj = _S0.sum(axis=0) + _I0.sum(axis=0) + _R0.sum(axis=0)
lambda_work = 0.5 * model.beta * I_ji_sumj / N_ji_sumj

print(lambda_home, _lambda_home)
print(lambda_work, _lambda_work)

M = S.shape[0]
lambda_ij = np.zeros_like(S)
for i in range(M):
    lambda_ij[i] = lambda_home[i] + lambda_work
print(lambda_ij, _lambda_ij)

[[63.56 32.58 23.86]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]]
[0.66666667 0.         0.        ] [0.66666667 0.         0.        ]
[0.49805447 0.27705628 0.27906977] [0.49805447 0.27705628 0.27906977]
[[1.16472114 0.94372294 0.94573643]
 [0.49805447 0.27705628 0.27906977]
 [0.49805447 0.27705628 0.27906977]] [[1.16472114 0.94372294 0.94573643]
 [0.49805447 0.27705628 0.27906977]
 [0.49805447 0.27705628 0.27906977]]


In [4]:
result = model.run_simulation()

## Show results

In [5]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('figure', dpi=200)

### Epidemic curve

In [6]:
figure = plt.figure(figsize=(3.5,2))
plt.plot(result['t'], result['S_total'], label='S', color='purple')
plt.plot(result['t'], result['I_total'], label='I', color='firebrick')
plt.plot(result['t'], result['R_total'], label='R', color='k')
plt.legend(frameon=False, loc='center right')
plt.xlabel("time")
plt.ylabel("compartment")
plt.show()
print(result['I_total'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[0.18181818181818182, 0.24848484848484848, 0.11969696969696969, 0.024242424242424242, 0.004545454545454545, 0.0, 0.0]


### Arrival times

The arrival time of the disease in the subpopulations

In [7]:
N_meas = 1000
all_frank_I = np.zeros_like(result['I_total'])
all_frank_R = np.zeros_like(result['R_total'])
for meas in range(N_meas):
    result = model.run_simulation()
    all_frank_I += np.array(result['I_total']) / N_meas
    all_frank_R += np.array(result['R_total']) / N_meas

In [8]:
from CommuteSIR.commute_sir import CommuteSIR

_beta = 4.
_mu = 2.

_F = np.array([
            [ 8, 2, 1 ],
            [ 4, 6, 0 ],
            [ 3, 0, 5 ],
        ],dtype=float)

model_kwargs = {
                'flux_matrix': _F,
                'population_numbers': np.array([360.,180.,120.]),
                'infection_rate': _beta,
                'recovery_rate': _mu,
                }
init_kwargs = {
                    'outbreak_location': 0,
                    'i0': 120,
                    'initial_condition_is_integer_number': True
            }

model2 = CommuteSIR(**model_kwargs)
model2.set_initial_conditions(**init_kwargs)
t = np.linspace(0,T_max,int(T_max/dt_save)+1)
all_ben_I = np.zeros_like(t)
all_ben_R = np.zeros_like(t)
for meas in range(N_meas):
    res = model2.simulate(t,dt=0.1)
    res = res.reshape(model2.y_shape + [len(t)])
    all_ben_I += res[:,:,1,:].sum(axis=0).sum(axis=0) / N_meas
    all_ben_R += res[:,:,2,:].sum(axis=0).sum(axis=0) / N_meas

In [9]:
figure = plt.figure(figsize=(3.5,2))
plt.plot(t, all_frank_I, label='I', color='firebrick')
plt.plot(t, all_ben_I/_Npop_home.sum(), ':', lw=2,label='I', color='k')
print(120/(360+180+120))
plt.legend(frameon=False, loc='center right')
plt.xlabel("time")
plt.ylabel("compartment")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.18181818181818182


In [10]:
figure = plt.figure(figsize=(3.5,2))
plt.plot(t, all_frank_R, label='R', color='firebrick')
plt.plot(t, all_ben_R/_Npop_home.sum(), ':', lw=2,label='I', color='k')
plt.legend(frameon=False, loc='center right')
plt.xlabel("time")
plt.ylabel("compartment")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …